<img align="left" src = https://project.lsst.org/sites/default/files/Rubin-O-Logo_0.png width=250 style="padding: 10px"> 
<p><p><p><p>
<b>Introduction to Rubin Image Services</b> <br>
Contact author: <i>Leanne Guy</i> <br>
Last verified to run: <i>2023-01-13</i> <br>
LSST Science Piplines version: Weekly <i>2022_24</i> <br>
Container Size: <i>medium</i> <br>
Targeted learning level: <i>intermediate</i> <br>

In [ ]:
# %load_ext pycodestyle_magic
# %flake8_on
# import logging
# logging.getLogger("flake8").setLevel(logging.FATAL)

**Skills:** Learn how to use Rubin Images Services to query and retrieve images. Learm how to  and image cutouts.

**LSST Image Data Products:** Single-epoch Images, Deep Coadds, 

**Packages:** lsst.rsp.get_tap_service, lsst.rsp.retrieve_query, pyvo

**Credit:** This tutorial was developed for DP0.2 by Leanne Guy.

**Get Support:**
Find DP0-related documentation and resources at <a href="https://dp0-1.lsst.io">dp0-1.lsst.io</a>. Questions are welcome as new topics in the <a href="https://community.lsst.org/c/support/dp0">Support - Data Preview 0 Category</a> of the Rubin Community Forum. Rubin staff will respond to all questions posted there.

## 1. Introduction

Image Services is the term used to describe the collection of Rubin services used for the discovery, description, access, and retrieval of LSST image data products. Rubin Observatory has adopted 'VO First' approach for services, meaning that wherever possible, IVOA-standard interfaces will be adopted. 
The Virtual Observatory (VO) is the vision that astronomical datasets and services should work together as a whole. The [International Virtual Observatory Alliance (IVOA)](ivoa.net) is an organisation that debates and agrees the technical standards that are needed to make the VO possible.

Tutorial 02, Catalog Queries with TAP, introduced the first Rubin IVOA-compliant service to be deployed as part of DP0.1 for accessing table data and metadata. The Rubin TAP service returns structured information about LSST data products.  In this tutorial we will introduce two new IVOA-compliant services that have been deployed as part of DP02 for accessing images and creating image cutouts.

Goals of this tutorial

* Introduce the IVOA ObsCore data model and the ObsTAP service and understand their use
* Search for and retrieve LSST image data products 
* Use the Rubin Image Cutout Service to create and retrieve cutouts around observations of interet

### 1.1 Package Imports

In [1]:
# Import general python packages
import numpy as np
import re
import pandas
from pandas.testing import assert_frame_equal
import uuid
import requests
import warnings

# Import the Rubin TAP service utilities
from lsst.rsp import get_tap_service, retrieve_query

# Science Pipelines imports
from lsst.daf.butler import Butler, DatasetType, CollectionType
import lsst.geom as geom
import lsst.resources
import lsst.geom as geom
import lsst.afw.image as afwImage
from lsst.afw.image import Image, ImageF
from lsst.afw.image.exposure import Exposure, ExposureF
import lsst.afw.display as afwDisplay

# Plotting with MPL
import matplotlib.pyplot as plt

# Import the Rubin TAP service utilities
from lsst.rsp import get_tap_service
from lsst.rsp import get_tap_service, retrieve_query
from lsst.rsp.utils import get_access_token

# PyVO packages
import pyvo
from pyvo.dal.adhoc import DatalinkResults, SodaQuery

# Astropy
from astropy import units as u
from astropy.units import UnitsWarning
from astropy.coordinates import SkyCoord
from astropy.io import fits
from astropy.time import Time
from astropy.utils.data import download_file
from astropy.visualization import  ZScaleInterval, AsinhStretch
from astropy.wcs import WCS       
from astropy.visualization import simple_norm, imshow_norm
from astropy.visualization import ImageNormalize,  ZScaleInterval
from astropy.visualization.stretch import SinhStretch, LinearStretch, SqrtStretch

# Holoviz for interactive visualization
import bokeh
from bokeh.io import output_file, output_notebook, show
from bokeh.layouts import gridplot
from bokeh.models import ColumnDataSource, CDSView, GroupFilter, HoverTool
from bokeh.plotting import figure
from bokeh.transform import factor_cmap
import holoviews as hv
from holoviews import streams, opts
from holoviews.operation.datashader import rasterize

### 1.2 Define Functions and Parameters

In [2]:
# Set the maximum number of rows to display from pandas
pandas.set_option('display.max_rows', 20)

In [3]:
#Set the DP0.2 config and collection, and instantiate a butler.
butler = Butler('dp02', collections='2.2i/runs/DP0.2')
registry = butler.registry

In [4]:
# The version of the LSST science piopelines should match the verified version listed at the start of the notebook
! echo ${IMAGE_DESCRIPTION}
! eups list lsst_distrib

Recommended (Weekly 2023_07)
   gdf42428520+759eb90f97 	current w_2023_07 setup


## 2 IVOA services 

Understanding the ivoa ObsCore table and its use .....

We will take a look at the LSST ObsCore Data Model (ObsCoreDM) and how to query it using the ObsTAP service. 
We will then look at how to download data products.

ObsTAP, from ObsCore 1.1 or later (as an initial priority), with SIAv2 (also ObsCore-based and serving equivalent data) later

As of DP0.2 there is a new schema (table collection) called \"ivoa\", which contains a table called ivoa.ObsCore. 
The IVOA-defined obscore table contains generic metadata for datasets held at the IDF. 
The table is accessible via ADQL queries via a TAP endpoint. The mechanism for locating images from obsevations is to make a TAP query against the ObsCore schema.

<br>
Here are some definitions to help you understand the contents of the ivoa schema. 

* `VO` - Vitrual Observatory - the vision that astronomical datasets and other resources should work as a seamless whole. Rubin is one of many projects and data centres worldwide who are are working towards this goal. Rubin has adopted a \"VO first strategy\".
* `IVOA` - International Virtual Observatory Alliance, ivoa.net. An organization that debates and agrees the technical standards that are needed to make the VO possible
* `ObsCore` - 
* `ObsTAP` - An IVOA standare
* `VOTable` - https://www.ivoa.net/documents/VOTable/ -- VOTable format is an XML standard for the interchange of data represented as a set of tables.
* `DataLink` - An IVOA standard to link from metadata about a dataset to the dataset itself, as well as other related data and services that can operate on that data

### 3.1 The Rubin ObsCore table
Let's take a look at how to query the ObsCore table. We will start by looking at the table collections (schemas) present.  In addition to the DP0.1 and DP0.2 catalog and TAP schema that were present in DP0.1, you will now see a new schema called \"ivoa\".

We will look at the ObsCore Data Model and explore some of the most common informatin fileds that people will want to use. 

In [31]:
service = get_tap_service()

In [7]:
query = """
SELECT * FROM tap_schema.schemas
"""
results = service.search(query).to_table()
results

description,schema_index,schema_name,utype
str512,int32,str64,str512
Data Preview 0.1 includes five tables based on the DESC's Data Challenge 2 simulation of 300 square degrees of the wide-fast-deep LSST survey region after 5 years. All tables contain objects detected in coadded images.,2,dp01_dc2_catalogs,
"Data Preview 0.2 contains the image and catalog products of the Rubin Science Pipelines v23 processing of the DESC Data Challenge 2 simulation, which covered 300 square degrees of the wide-fast-deep LSST survey region over 5 years.",0,dp02_dc2_catalogs,
ObsCore v1.1 attributes in ObsTAP realization,1,ivoa,
A TAP-standard-mandated schema to describe tablesets in a TAP 1.1 service,100000,tap_schema,
UWS Metadata,120000,uws,


Let's use the TAP service to look at the tables in the ivoa table collection.  

In [8]:
query = """
SELECT * FROM tap_schema.tables where schema_name like 'ivoa' order by table_index ASC
"""
result = service.search(query).to_table()
result

description,schema_name,table_index,table_name,table_type,utype
str512,str512,int32,str64,str8,str512
Observation metadata in the ObsTAP relational realization of the IVOA ObsCore data model,ivoa,0,ivoa.ObsCore,table,


The ivoa schema contains 1 table called \"ivoa.ObsCore\". This table holds the observation metadata for all images in the DP0.2 dataset. Let's look a the main concepts in the ObsCore data model. 

In [15]:
query = """SELECT column_name, unit, ucd, description, utype
  FROM tap_schema.columns
  WHERE table_name = 'ivoa.ObsCore'
"""
result = service.search(query).to_table()
result

column_name,unit,ucd,description,utype
str64,str64,str64,str512,str512
access_format,,meta.code.mime,Content format of the dataset,Access.format
access_url,,meta.ref.url,URL used to access dataset,Access.reference
calib_level,,meta.code;obs.calib,"Calibration level of the observation: in {0, 1, 2, 3, 4}",ObsDataset.calibLevel
dataproduct_subtype,,meta.code.class,Data product specific type,ObsDataset.dataProductSubtype
dataproduct_type,,meta.code.class,Data product (file content) primary type,ObsDataset.dataProductType
em_max,m,em.wl;stat.max,stop in spectral coordinates,Char.SpectralAxis.Coverage.Bounds.Limits.HiLimit
em_min,m,em.wl;stat.min,start in spectral coordinates,Char.SpectralAxis.Coverage.Bounds.Limits.LoLimit
em_res_power,,spect.resolution,Value of the resolving power along the spectral axis (R),Char.SpectralAxis.Resolution.ResolPower.refVal
em_xel,,meta.number,Number of elements along the spectral axis,Char.SpectralAxis.numBins


The ObsCore data model that we see above results from analysis carried out by the IVOA to support global data discovery and accessibility. Many of the columns are a mandatory fields of the Observation Core Components data model including their name, recommended units, data type and designation.

The IVOS maintains a registry of all VO compliant services. We can interrogate this registry to find all the Tap services that support the ObsCoreDM. Valid registry keywords incluide: author, datamodel, ivoid, keywords, servicetype, spatial, spectral, temporal, ucd, waveband.

In [28]:
for vo_service in pyvo.regsearch(datamodel='obscore'):
    print(vo_service['ivoid'])  

ivo://archive.stsci.edu/caomtap
ivo://astro.ucl.ac.uk/tap
ivo://astron.nl/tap
ivo://asu.cas.cz/tap
ivo://au.csiro/atoavo/tap
ivo://au.csiro/casda/tap
ivo://bira-iasb/tap
ivo://byu.arvo/tap
ivo://cadc.nrc.ca/argus
ivo://cefca/j-plus/j-plus-dr1
ivo://cefca/j-plus/j-plus-dr2
ivo://cefca/j-plus/j-plus-dr3
ivo://cefca/minijpas/minij-pas-pdr201912
ivo://chivo/tap
ivo://eso.org/tap_obs
ivo://fai.kz/tap
ivo://fu-berlin.planet.hrsc/tap
ivo://ia2.inaf.it/hosted/laurino2011/tap
ivo://ia2.inaf.it/tap
ivo://jive.eu/tap
ivo://jvo/alma
ivo://jvo/isas/darts/akari/akari-irc_catalogue_allsky_astflux_1.0
ivo://jvo/isas/darts/akari/akari-irc_spectrum_pointed_diffuseskypatch_1.0
ivo://jvo/isas/darts/akari/akari-irc_spectrum_pointed_galpn_1.0
ivo://jvo/isas/darts/akari/akari-irc_spectrum_pointed_slitlessmir_spectrum_1.0
ivo://jvo/isas/darts/halca/halca_vsop_correlated_data
ivo://jvo/isas/darts/halca/halca_vsop_survey_program_data
ivo://jvo/isas/darts/hitomi/hitomaster_v1
ivo://jvo/nobeyama
ivo://jvo/subaru/

The Rubin ObsCore table holds the observation metadata for all images in the DP0.2 dataset. Let's explore the contents of the ObsCore table. 

In [35]:
query = """ 
SELECT dataproduct_type, COUNT(*) 
FROM ivoa.ObsCore 
GROUP BY dataproduct_type
"""
result = service.search(query).to_table()
result

dataproduct_type,COUNT
str128,int64
image,8475974


There are 8475974 entries for the DP0.2 dataset. The model defines a data product type attribute. This is the observation product that a user will query for an retrieve, e.g \"image\", \"spectrum\". There is currently only one data product type supported in the Rubin ObsCore DM -- \"image\". 

In addition to the data product type, there are five unique data product subtypes defined in the ObsCore table. The data product subtype allows us to be more specific about data product type. Let's take a look at the data products defined in the Rubin ObsCore DM. 

Note: [dp0-2.lsst.io](https://dp0-2.lsst.io/data-products-dp0-2/index.html) lists many more data product types that are not listed here. For example, "deepCoadd" is not listed and you will not find any deepCoadd data product in the ObsCore table. Need to investigate. 

In [38]:
query = """ 
SELECT dataproduct_subtype, COUNT(*) 
FROM ivoa.ObsCore 
GROUP BY dataproduct_subtype
"""
result = service.search(query).to_table()
result

dataproduct_subtype,COUNT
str64,int64
lsst.calexp,2805017
lsst.deepCoadd_calexp,46158
lsst.goodSeeingCoadd,46158
lsst.goodSeeingDiff_differenceExp,2707834
lsst.raw,2870807


The \"calib_level\" column defines the calibration level of the data product and tells the user the amount of calibration processing that was applied to create the data product. The calib_level column cannot be null for any entry in the ObsCore table. The following goves the defined calib_levels and their relationship to the LSST data products. 

* `0` - raw instrumental data, possibly in proprietary internal provider format
* `1` - instrumental data that may or may mot be calibrated in standard format -- lsst.raw
* `2` - science-ready calibrated data with instrument signature removed -- lsst.calexp
* `3` - enhanced science-ready data products, e.g mosaics, coadds, difference images -- lsst.deepCoadd_calexp, lsst.goodSeeingDiff_differenceExp, lsst.goodSeeingCoadd

In [41]:
query = """ SELECT calib_level, COUNT(*) 
FROM ivoa.ObsCore 
GROUP BY calib_level
"""
result = service.search(query).to_table()
result

calib_level,COUNT
int32,int64
1,2870807
2,2805017
3,2800150


In [42]:
query = """ SELECT dataproduct_subtype, COUNT(*) 
FROM ivoa.ObsCore 
where calib_level = '1'
GROUP BY dataproduct_subtype
"""
result = service.search(query).to_table()
result

dataproduct_subtype,COUNT
str64,int64
lsst.raw,2870807


The ObsCore model also defines a region with the \"s_region\" column expressed in the ICRS frame. We can use this attribute to select all DP0.2 image data products of type \"deepCoadd_calexp\" in a given defined region. Let's define a point of interest and a search radius. 

In [44]:
center_coords = SkyCoord(55.74673760481304, -32.286155241413624, frame='icrs', unit='deg')
search_radius = 10*u.deg

In [45]:
query = """SELECT * FROM ivoa.ObsCore 
WHERE dataproduct_type = 'image'
AND obs_collection = 'LSST.DP02' 
AND dataproduct_subtype = 'lsst.deepCoadd_calexp'
AND CONTAINS(POINT('ICRS', 55.74673760481304, -32.286155241413624), s_region)=1 
"""
results = service.search(query).to_table()
results

access_format,access_url,calib_level,dataproduct_subtype,dataproduct_type,em_max,em_min,em_res_power,em_xel,facility_name,instrument_name,lsst_band,lsst_detector,lsst_filter,lsst_patch,lsst_tract,lsst_visit,o_ucd,obs_collection,obs_id,obs_publisher_did,pol_xel,s_dec,s_fov,s_ra,s_region,s_resolution,s_xel1,s_xel2,t_exptime,t_max,t_min,t_resolution,t_xel,target_name
,,,,,m,m,,,,,,,,,,,,,,,,deg,deg,deg,,arcsec,,,s,d,d,s,,
str128,object,int32,str64,str128,float64,float64,float64,int64,str128,str128,str10,int64,str10,int64,int64,int64,str32,str128,str128,str256,int64,float64,float64,float64,str512,float64,int64,int64,float64,float64,float64,float64,int64,str32
application/x-votable+xml;content=datalink,https://data.lsst.cloud/api/datalink/links?ID=butler%3A//dp02/14e949ec-59a0-4e9a-99c3-dbc74e904fe8,3,lsst.deepCoadd_calexp,image,1.06e-06,9.7e-07,--,--,Rubin-LSST,,y,--,,17,4431,--,phot.count,LSST.DP02,DC2-4431-17,,--,-32.205663176664345,0.3299793994195083,55.65214108466622,POLYGON ICRS 55.514085 -32.322253 55.790197 -32.322253 55.789845 -32.088924 55.514437 -32.088924,--,--,--,--,--,--,--,--,
application/x-votable+xml;content=datalink,https://data.lsst.cloud/api/datalink/links?ID=butler%3A//dp02/b876fbf2-9bbb-427d-8e86-e3c272054d34,3,lsst.deepCoadd_calexp,image,9.22e-07,8.18e-07,--,--,Rubin-LSST,,z,--,,17,4431,--,phot.count,LSST.DP02,DC2-4431-17,,--,-32.205663176664345,0.3299793994195083,55.65214108466622,POLYGON ICRS 55.514085 -32.322253 55.790197 -32.322253 55.789845 -32.088924 55.514437 -32.088924,--,--,--,--,--,--,--,--,
application/x-votable+xml;content=datalink,https://data.lsst.cloud/api/datalink/links?ID=butler%3A//dp02/f9ba9b77-3e1d-409c-b7f1-1be7e42ff790,3,lsst.deepCoadd_calexp,image,6.91e-07,5.52e-07,--,--,Rubin-LSST,,r,--,,17,4431,--,phot.count,LSST.DP02,DC2-4431-17,,--,-32.205663176664345,0.3299793994195083,55.65214108466622,POLYGON ICRS 55.514085 -32.322253 55.790197 -32.322253 55.789845 -32.088924 55.514437 -32.088924,--,--,--,--,--,--,--,--,
application/x-votable+xml;content=datalink,https://data.lsst.cloud/api/datalink/links?ID=butler%3A//dp02/3b7eb0b6-71cc-4b09-96dd-17293d57bf84,3,lsst.deepCoadd_calexp,image,5.52e-07,4.02e-07,--,--,Rubin-LSST,,g,--,,17,4431,--,phot.count,LSST.DP02,DC2-4431-17,,--,-32.205663176664345,0.3299793994195083,55.65214108466622,POLYGON ICRS 55.514085 -32.322253 55.790197 -32.322253 55.789845 -32.088924 55.514437 -32.088924,--,--,--,--,--,--,--,--,
application/x-votable+xml;content=datalink,https://data.lsst.cloud/api/datalink/links?ID=butler%3A//dp02/bcaafd06-fa5e-4b32-bf2e-351c23a4da83,3,lsst.deepCoadd_calexp,image,4e-07,3.3e-07,--,--,Rubin-LSST,,u,--,,17,4431,--,phot.count,LSST.DP02,DC2-4431-17,,--,-32.205663176664345,0.3299793994195083,55.65214108466622,POLYGON ICRS 55.514085 -32.322253 55.790197 -32.322253 55.789845 -32.088924 55.514437 -32.088924,--,--,--,--,--,--,--,--,
application/x-votable+xml;content=datalink,https://data.lsst.cloud/api/datalink/links?ID=butler%3A//dp02/20d28216-534a-4102-b8a7-1c7f32a9b78c,3,lsst.deepCoadd_calexp,image,8.18e-07,6.91e-07,--,--,Rubin-LSST,,i,--,,17,4431,--,phot.count,LSST.DP02,DC2-4431-17,,--,-32.205663176664345,0.3299793994195083,55.65214108466622,POLYGON ICRS 55.514085 -32.322253 55.790197 -32.322253 55.789845 -32.088924 55.514437 -32.088924,--,--,--,--,--,--,--,--,


We see that there are 6 deepCoadd_calexp images, one for each filter. Alternatively, kokwing the patch and tract,  can query directly for the same images and specify a filter. 

In [46]:
query = """SELECT * FROM ivoa.ObsCore 
WHERE dataproduct_type = 'image'
AND obs_collection = 'LSST.DP02' 
AND dataproduct_subtype = 'lsst.deepCoadd_calexp'
AND lsst_tract = 4431
AND lsst_patch = 17
AND lsst_band = 'i'
"""
results = service.search(query).to_table()
results

access_format,access_url,calib_level,dataproduct_subtype,dataproduct_type,em_max,em_min,em_res_power,em_xel,facility_name,instrument_name,lsst_band,lsst_detector,lsst_filter,lsst_patch,lsst_tract,lsst_visit,o_ucd,obs_collection,obs_id,obs_publisher_did,pol_xel,s_dec,s_fov,s_ra,s_region,s_resolution,s_xel1,s_xel2,t_exptime,t_max,t_min,t_resolution,t_xel,target_name
,,,,,m,m,,,,,,,,,,,,,,,,deg,deg,deg,,arcsec,,,s,d,d,s,,
str128,object,int32,str64,str128,float64,float64,float64,int64,str128,str128,str10,int64,str10,int64,int64,int64,str32,str128,str128,str256,int64,float64,float64,float64,str512,float64,int64,int64,float64,float64,float64,float64,int64,str32
application/x-votable+xml;content=datalink,https://data.lsst.cloud/api/datalink/links?ID=butler%3A//dp02/20d28216-534a-4102-b8a7-1c7f32a9b78c,3,lsst.deepCoadd_calexp,image,8.18e-07,6.91e-07,--,--,Rubin-LSST,,i,--,,17,4431,--,phot.count,LSST.DP02,DC2-4431-17,,--,-32.205663176664345,0.3299793994195083,55.65214108466622,POLYGON ICRS 55.514085 -32.322253 55.790197 -32.322253 55.789845 -32.088924 55.514437 -32.088924,--,--,--,--,--,--,--,--,


The image UUID that we extracted from the Butler above is part of the access url field. We can query on that as well.  We passed a single UUID (in the first query) so there must be 1 result only (but there are zero)

In [ ]:
query = """
SELECT * FROM ivoa.ObsCore WHERE access_url like '%20d28216-534a-4102-b8a7-1c7f32a9b78c' 
"""
results = service.search(query).to_table()
assert len(results) == 1  
results

### 3.2 Accessing Images

Now that we know how to query the ObsCore table to find images, lets see how to access, manipulate and download them

In [ ]:
# Now lets retrieve the deep coadd from previously as well as one of the visit images that 
query = """SELECT access_format, access_url, dataproduct_subtype, lsst_patch, lsst_tract, lsst_band, s_ra, s_dec
FROM ivoa.ObsCore 
WHERE dataproduct_type = 'image'
AND obs_collection = 'LSST.DP02' 
AND dataproduct_subtype = 'lsst.deepCoadd_calexp'
AND lsst_tract = 4431
AND lsst_patch = 17
AND lsst_band = 'i'
"""
results = service.search(query)
results.to_table().show_in_notebook()

Take a look at the columns \"access_format\" and \access_url\". A access_url is provided for each image in a row from the ObsCore query.  <rsp-base-url>/api/datalink/links?ID=<id> where <rsp-base-url> is the base URL of the Rubin Science Platform and <id> is a UUID for that image in Butler. Recall previously that we were able to query by UUID. \"access_format\" tells us that the format is DataLinks. We can get the DataLinks URL as follows: 

In [ ]:
dataLinkUrl = results[0].getdataurl()
f"Datalink link service url: {dataLinkUrl}"

Before proceeding we need to extract the session authentiction for reuse. Explain more about this

In [ ]:
auth_session = service._session

We call call PyVo's DatalinkResults with the DataLink URL for the image we want to retrieve. This returns a list matching records, each record containoing a set of metadata describing the record.

In [ ]:
dl_results = DatalinkResults.from_result_url(dataLinkUrl,session=auth_session)
f"{dl_results.status}"

In [ ]:
dl_results.to_table().show_in_notebook()

In the table above we see that two records are returned for our image. The first is a signed URL for the image (Primary image or observation data file) that is valid for an hour. <ADD some text explaingin where the image is stored and the URL>. The second is a link to the cutout service. 

In [ ]:
image_url = dl_results.getrecord(0).get('access_url')
f"{image_url}"

We can plot the image by making an Exposure object directly from the URL. This plot looks identical to the plot of the same image retrived via the Butler

In [ ]:
new_coadd = ExposureF(image_url)
plotImage(new_coadd)

In [ ]:
# We can open the URL and take look at the headers 
hdulist = fits.open(image_url)
for hdu in hdulist:
    print(hdu.name)

In [ ]:
# We can also download the file
filename = download_file(image_url)
assert os.path.isfile(filename)
f"{filename}"

## 4 Image cutout service 

The Rubin Image Cutout Serivce is implemented using the [IVOA SODA](https://ivoa.net/documents/SODA/20170517/REC-SODA-1.0.html) standard. SODA (Server-side Operations for Data Access) is a low-level data access capability or server side data processing that can act upon data files, performing various kinds of operations, such as filtering/subsection, transformations, pixel operations, and applying functions to the data.

Need the Butler UUID of an image 

The initial implementation of the image cutout service will only return FITS files

Initial implementation supports 
CIRCLE and POLYGON.  POS=RANGE not implemented - check!

The initial version of the cutout service will only support a single ID parameter and a single stencil parameter.
The ID parameter must be a UUID assigned by the Butler and uniquely identifying a source image

The Image cutout service is based on SODA 1.0. 
Examples: 
https://github.com/astropy/pyvo/blob/main/examples/images/ex_get_cutouts.py
https://github.com/astropy/pyvo/blob/main/examples/images/ex_casA_image_cat.py

## 4.1 Using PyVO's SodaQuery object

Introduction to SODA

SODA, describe the acronym is a ..... , describe service
The Rubin SODA service is to support performing cutouts from the collected LSST image data.

Only synchronouse queries of the cutout service run currently. An asynchronous quert support may be introduced in the future

SODA needs the Butler UUID. Specifically, the URL is similar to https://data-int.lsst.cloud/api/cutout/sync?id=8a953c0321bd4878bfa694dbf628ea81&circle=53.13925%20-34.0215%200.0105 (the id parameter is the Butler UUID, and the remaining parameters are the cutout request, and there's a POST version as well and an async version following the SODA standard).

Now lets look at the second result in the DataLink table above. This record provides informaitn about the cutout service. We will use this to get a cutout by using SodaQuery 

Note:  add some description and an introduction here as to what SODA and a SODA service is. 

Prepare a query to the SODA service using the DataLink results from above and passing the session authorization token. 

In [ ]:
sq = SodaQuery.from_resource(dl_results, dl_results.get_adhocservice_by_id("cutout-sync"), 
                             session=auth_session)

Now define a a circle cutout centered on the galaxy cluster 

In [ ]:
sphereRadius = 0.03* u.deg
sq.circle = (spherePoint.getRa().asDegrees()* u.deg,
             spherePoint.getDec().asDegrees()*u.deg, 
             sphereRadius)
f"Circle around point {spherePoint} of radius {sphereRadius}"

In [ ]:
# Which shapes / stencils are supported currently
sq.

In [ ]:
#help(sq)

Now create the cutout 

In [ ]:
sodaCutout = os.path.join(os.getenv('HOME'), 'DATA/soda-cutout.fits')
with open(sodaCutout, 'bw') as f:
    f.write(sq.execute_stream().read())

In [ ]:
# Display the cutout
plotImage(ExposureF(sodaCutout))

We can see that the image cutout created by the cutout service looks identical to the one we obtained via the Butler

### 4.2 Using the SODA endpoint

We construct the SODA endoint as follows. This is similar to endpoints for TAP 

In [ ]:
host=os.getenv("EXTERNAL_INSTANCE_URL")
SODA_URL="{}/api/image/soda/sync".format(host)
ID='default.calexp.r'
print(SODA_URL)

## 5. Exercises to the user 

* Explore the obscore data model for another archive
* Explore some of te other attributes defined in the ObsCore DM table - how might you use them to query and retrieve data products of interest for science
* Get the UUID for all the visits that comprise the deepCoadd, get a cutout of the galaxy cluster and display them in a grid of images 
* Do a reprocessing on a cutout
* Find an interesting object and use the cutout service to get cutouts of the object in all bands

## 6. References 

Rubin technotes
*  [DMTN-208](dmtn-208.lsst.io): RSP image cutout service implementation strategy
*  [DMTN-238](dmtn-238.lsst.io): RSP DataLink service implementation strategy
*  https://dmtn-139.lsst.io/v/DM-22746/index.html , section 7.4 (not published yet)


Relevant IVOA standards documents 
*  IVOA Table Access Protocol (TAP) https://www.ivoa.net/documents/TAP/